# [동적 모델 설계 구현]

-	사용 모듈 : nn.ModuleList
-	특징 : 일반 list 로는 pytorch 에서 Layer 인식 안된 ==> 대안 : ModuleList

In [5]:
# -------------------------------------------------
# 모델 설계
# -	입력층
# - 은닉층  <=== 유동적 0개 ~ N 개 : 모델 인스턴스 생성 시 매개 변수 전달인자
# - 출력층 

In [6]:
# 모듈 로딩 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader					# 데이터셋 관련

from torchmetrics.classification import *						# 모델성능 지표 관련
from torchinfo import summary									# 모델구조 및 정보 관련

from sklearn.model_selection import train_test_split			# 학습용 & 테스트용 데이터셋 분리
from sklearn.preprocessing import MinMaxScaler,StandardScaler	# 연속형 피쳐 스케일링
from sklearn.preprocessing import OneHotEncoder,LabelEncoder	# 범주형 피쳐 타겟 인코딩

In [ ]:
# 동적 모델 클래스 정의 (2) - 은닉층 개수 및 뉴런 개수 동적인 모델 -------------------------
# 
class MyModel(nn.Module):

	def __init__(self, in_in, out_out, h_in, h_cnt) :
		# 부모 클래스
		super().__init__()

		# 자식 클래스 의 인스턴스 속성 설정
		self.input_layer = nn.Linear(in_in, h_in)
		self.h1_layer = nn.ModuleList([nn.Linear(h_in,h_in) for _ in range(h_cnt)])
		self.output_layer = nn.Linear(h_in, out_out)



	def forward(self,x):

		y = F.relu(self.input_layer(x))

		for linear in self.h1_layer:
			y = F.relu(linear(y))

		
		return self.output_layer(y)

In [12]:
from torchinfo import summary

In [14]:
# 모델 인스터스 생성
# in_in = 3
# out_out = 1
# hd_in = 5
# hd_cut =1
m1 = MyModel(3,1,50,1)

m1(torch.FloatTensor([[1,2,3]]))		# 샘플수 , 피쳐수

tensor([[-0.1037]], grad_fn=<AddmmBackward0>)

In [15]:
summary(m1, input_size=(100,3))

Layer (type:depth-idx)                   Output Shape              Param #
MyModel                                  [100, 1]                  --
├─Linear: 1-1                            [100, 50]                 200
├─ModuleList: 1-2                        --                        --
│    └─Linear: 2-1                       [100, 50]                 2,550
├─Linear: 1-3                            [100, 1]                  51
Total params: 2,801
Trainable params: 2,801
Non-trainable params: 0
Total mult-adds (M): 0.28
Input size (MB): 0.00
Forward/backward pass size (MB): 0.08
Params size (MB): 0.01
Estimated Total Size (MB): 0.09

In [11]:
m2 = MyModel(3,3,10,10)

m2(torch.FloatTensor([[1,2,3]]))		# 샘플수 , 피쳐수
m2

MyModel(
  (input_layer): Linear(in_features=3, out_features=10, bias=True)
  (h1_layer): ModuleList(
    (0-9): 10 x Linear(in_features=10, out_features=10, bias=True)
  )
  (output_layer): Linear(in_features=10, out_features=3, bias=True)
)

In [23]:
# 은닉층 개수 및 뉴런 개수 동적 모델(2) > 은닉층 개수와 뉴런 개수 동적인 모델 
# in_in : 입력층의 입력 in_out : 입력층의 출력 / 다음층의 입력
# out_out : 출력층의 출력 h_outs = [] : 은닉층의 출력 리스트
class DynamicModel(nn.Module):
   def __init__(self, in_in, in_out ,out_out, h_outs = []) :
      # 부모 클래스 생성
      super().__init__()
   
   # 자식클래스의 인스턴스 속성 설정
      self.input_layer = nn.Linear(in_in, in_out)

      # 은닉층 여러개 생성
      self.h1_layer = nn.ModuleList()
      for idx in range(len(h_outs)):
         h_in = h_outs[idx-1] if idx else in_out
         h_out = h_outs[idx]
         self.h1_layer.append(nn.Linear(h_in, h_out))
      
      # 출력층 생성
      self.output_layer = nn.Linear(h_outs[-1] if len(h_outs) else in_out, out_out)

   
   # 순전파 학습 메서드
   def forward(self,x):

      y = F.relu(self.input_layer(x))

      for linear in self.h1_layer:
         y = F.relu(linear(y))

      return self.output_layer(y)



In [ ]:
# # 동적 모델 클래스 정의 (2) - 은닉층 개수 및 뉴런 개수 동적인 모델 -------------------------
# # in_in : 입력층의 입력  in_out 		: 입력층의 출력/ 다음층의 입력
# # out_out : 출력층의 출력 h_outs = []	: 은닉층의 출력리스트

# class DyanmicMyModel(nn.Module):

# 	def __init__(self, in_in, in_out,out_out, h_outs=[]) :
# 		# 부모 클래스
# 		super().__init__()

# 		# 자식 클래스 의 인스턴스 속성 설정
# 		self.input_layer = nn.Linear(in_in, h_in)
		
# 		# 은닉층 여러개 생성
# 		self.h1_layer = nn.ModuleList()
		
# 		for idx in range(len(h_outs)):
# 			h_in = h_outs[idx-1] if idx else in_out
# 			h_out = h_outs[idx]
# 			self.h1_layer.append( nn.Linear(h_in, h_out))

		
# 		# 출력층 생성
# 		self.output_layer = nn.Linear(h_outs[-1] if len(h_outs) else in_out, out_out)


# 	def forward(self,x):

# 		y = F.relu(self.input_layer(x))

# 		for linear in self.h1_layer:
# 			y = F.relu(linear(y))

		
# 		return self.output_layer(y)

In [24]:
# 모델 인스터스 생성
# in_in : 입력층의 입력   in_out : 입력층의 출력/ 다음층의 입력
# out_out : 출력층의 출력 h_outs =[] : 은닉층의 출력  리스트
m1 = DynamicModel(3,5,1)
m1(torch.FloatTensor([[1,2,3]]))		# 샘플수 , 피쳐수

summary(m1, input_size=(100,3))

Layer (type:depth-idx)                   Output Shape              Param #
DynamicModel                             [100, 1]                  --
├─Linear: 1-1                            [100, 5]                  20
├─Linear: 1-2                            [100, 1]                  6
Total params: 26
Trainable params: 26
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01

In [27]:
# 모델 인스터스 생성
# in_in : 입력층의 입력   in_out : 입력층의 출력/ 다음층의 입력
# out_out : 출력층의 출력 h_outs =[] : 은닉층의 출력  리스트
m1 = DynamicModel(3,10,3,[7,5])
m1(torch.FloatTensor([[1,2,3]]))		# 샘플수 , 피쳐수

summary(m1, input_size=(100,3))

Layer (type:depth-idx)                   Output Shape              Param #
DynamicModel                             [100, 3]                  --
├─Linear: 1-1                            [100, 10]                 40
├─ModuleList: 1-2                        --                        --
│    └─Linear: 2-1                       [100, 7]                  77
│    └─Linear: 2-2                       [100, 5]                  40
├─Linear: 1-3                            [100, 3]                  18
Total params: 175
Trainable params: 175
Non-trainable params: 0
Total mult-adds (M): 0.02
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.00
Estimated Total Size (MB): 0.02